### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [479]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

schools_path = os.path.join("Resources" , "schools_complete.csv")
students_path = os.path.join ("Resources", "students_complete.csv")

schools_data = pd.read_csv(schools_path)
students_data = pd.read_csv(students_path)

data_complete = pd.merge(students_data, schools_data, 
                how="left", 
                on=["school_name", "school_name"])\
                .rename(columns=lambda s:s.lower().replace(" ", "_"))  


,student_id,student_name,gender,grade,school_name,reading_score,math_score,school_id,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [583]:
total_schools = len(data_complete["school_name"].unique())
total_students = len(data_complete["student_id"].unique())
district_budget = data_complete.drop_duplicates('school_name')['budget'].sum()
math_average = round(data_complete["math_score"].mean())
reading_average = round(data_complete["reading_score"].mean())
students_passing_math = data_complete[data_complete["math_score"] >= 70].count()["student_id"]
students_passing_reading = data_complete[data_complete["reading_score"] >= 70].count()["student_id"]
percentage_students_passed_math = round((students_passing_math/total_students)*100,2)
percentage_students_passed_reading = round((students_passing_reading/total_students)*100,2)
overall_passing_rate = round(percentage_students_passed_math + percentage_students_passed_reading)/2

d = {"Summary":["Total Schools", "Total Students", "District Budget", "District Passing Rate"],
    " ":[total_schools, total_students, district_budget, overall_passing_rate]}

d2 = {"Summary": ["Average Grade", "Passing Students"],
      "Math": [math_average, percentage_students_passed_math],
      "Reading": [reading_average, percentage_students_passed_reading]}
district_summary = pd.DataFrame(d)
education_summary = pd.DataFrame(d2)

## District Summary

In [584]:
district_summary


,Summary,
0,Total Schools,15.0
1,Total Students,39170.0
2,District Budget,24649428.0
3,District Passing Rate,80.5


In [585]:
education_summary

,Summary,Math,Reading
0,Average Grade,79.00,82.00
1,Passing Students,74.98,85.81


74.98

In [493]:
district_score = pd.DataFrame({
    #"total_schools": [total_schools],
   # "total_students": [total_students],     
   # "total_budget": [district_budget],
   "math_average_score": [math_average],
   "reading_average_score": [reading_average],
   # "overall_passing_rate" : [overall_passing_rate],
    "%_math_passing":  [math_passing],
    "%_reading_passing": [reading_passing]
})
district_score

,math_average_score,reading_average_score,%_math_passing,%_reading_passing
0,79.0,82.0,"([74.98, 25.02], 0)",True 85.81 False 14.19 Name: reading_sc...


In [ ]:
district_summary = pd.DataFrame ({"Total":[len(data_complete["school_name"].unique()),
                                          len(data_complete["student_id"].unique()),
                                          data_complete.drop_duplicates('school_name')['budget'].sum(),
                                         (math_average + reading_average)/2]},\
                                 # index=["Schools", "Students", "Budget", "Overall Passing Rate"]
                                 #)
#district_score = pd.DataFrame({
   # "total_schools": [len(data_complete["school_name"].unique())],
    #"total_students": [len(data_complete["student_id"].unique())],     
    #"total_budget": [data_complete.drop_duplicates('school_name')['budget'].sum()],
   # "math_average_score": [data_complete["math_score"].mean()],
   # "reading_average_score": [data_complete["reading_score"].mean()],
    #"overall_passing_rate" : [(math_average + reading_average)/2],
    #"%_math_passing" :[(data_complete["math_score"] >= 70).value_counts()/total_students],
    #"%_reading_passing" : [(data_complete["reading_score"] >= 70).value_counts()/total_students],
#})
#district_summary

#reading_passing

#district =pd.DataFrame(pd.Series([
 #           len(data_complete["school_name"].unique()),
  #          len(data_complete["student_id"].unique()),
   #         data_complete.drop_duplicates('school_name')['budget'].sum()],
    #        index=["Number of Schools", "Number os Students", "District Budget"]
    #))

#district

In [472]:
schools_data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


## School Summary

In [473]:
schools_name = data_complete["school_name"].unique()
school_type = schools_data.set_index(["school_name"])["type"]   
total_students_per_school = data_complete.groupby(["school_name"]).mean()["size"]

budget_per_school = data_complete.groupby(["school_name"]).mean()["budget"]

budget_per_student = budget_per_school/total_students_per_school
math_average_per_school = round(data_complete.groupby(["school_name"]).mean()["math_score"],2)
math_average_per_school
reading_average_per_school = round(data_complete.groupby(["school_name"]).mean()["reading_score"],2)
budget_per_school.map("${:,.2f}".format)





school_name
Bailey High School       $3,124,928.00
Cabrera High School      $1,081,356.00
Figueroa High School     $1,884,411.00
Ford High School         $1,763,916.00
Griffin High School        $917,500.00
Hernandez High School    $3,022,020.00
Holden High School         $248,087.00
Huang High School        $1,910,635.00
Johnson High School      $3,094,650.00
Pena High School           $585,858.00
Rodriguez High School    $2,547,363.00
Shelton High School      $1,056,600.00
Thomas High School       $1,043,130.00
Wilson High School       $1,319,574.00
Wright High School       $1,049,400.00
Name: budget, dtype: object

In [367]:

#students passing math by schools
students_passing_math = data_complete[(data_complete["math_score"]>=70)]
students_passing_math_per_school = students_passing_math.groupby(["school_name"]).count()["student_id"]
percentage_passing_math_per_school = round((students_passing_math_per_school/total_students_per_school)*100,2)


students_passing_reading = data_complete[(data_complete["reading_score"]>=70)]
students_passing_reading_per_school = students_passing_reading.groupby(["school_name"]).count()["student_id"]
percentage_passing_reading_per_school = round((students_passing_reading_per_school/total_students_per_school)*100,2)
percentage_overall_passing_rate_per_school = round(percentage_passing_math_per_school+percentage_passing_reading_per_school)/2
school_summary = pd.DataFrame["schools_name", "school_type"]
school_summary

#sumary_df = pd.DataFrame({
   # "Authors_count": [len(df["Authors"].unique())],
   # "earliest_year": [df["Publication Year"].min()],     
   # "latest_year": [df["Publication Year"].max()],
  #  "total_reviews": [df["One Star Reviews"].sum() + df["Two Star Reviews"].sum() + df["Three Star Reviews"].sum() + df["Four Star Reviews"].sum() + df["Five Star Reviews"].sum()]
#})
#sumary_df

TypeError: 'type' object is not subscriptable

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [350]:
top_schools = data_complete.sort_values(by="percentage_overall_passing_rate_per_school")
top_schools

KeyError: 'percentage_overall_passing_rate_per_school'

In [347]:
top_schools

student_id                    39169
student_name              Zoe Meyer
gender                            M
grade                           9th
school_name      Wright High School
reading_score                    99
math_score                       99
school_id                        14
type                       District
size                           4976
budget                      3124928
dtype: object

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
